### **Importando as bibliotecas**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import TextLoader # Permite carregar arquivos de texto
from langchain_text_splitters import CharacterTextSplitter # Permite dividir o texto em partes menores

from langchain_pinecone import PineconeVectorStore # Permite armazenar vetores no Pinecone
from langchain.memory import ConversationBufferMemory

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

from langchain_groq import ChatGroq
from langchain import hub



### **Carregar as variáveis de ambientes**

In [4]:
load_dotenv(find_dotenv())
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


### **INSTANCIAR O BANCO DE DADOS DO PINECONE**

OBS: Devemos criar o index no pinecone cloud

In [5]:
index_name = "rag-demo"
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=COHERE_API_KEY
)

### **Criando a mémoria**

In [6]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\tavin\AppData\Local\Temp\ipykernel_9588\515166408.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


### **CARREGAMENTO/INGESTÃO DO DOCUMENTO**

In [ ]:
print("🔥Carregando os documentos...\n\n")
PATH_FILE = "data\mediumblog1.txt"
loader = TextLoader(PATH_FILE)
document = loader.load()


Carregando os documentos...




<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\tavin\AppData\Local\Temp\ipykernel_9588\3164062685.py:2: SyntaxWarning: invalid escape sequence '\m'
  PATH_FILE = "data\mediumblog1.txt"


### **FATIMANETO/SPLITTING DOS DOCUMENTOS**

In [ ]:
print("Spplitting the document into smaller chunks...")
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
docs = text_splitter.split_documents(document)
print(f"Total chunks: {len(docs)}\n\n")


### **Instanciar o banco de dados**

Após a execução deste comando, os dados estarão disponiveis no banco de dados cloud no Pinecone

In [23]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    index_name=index_name,
    embedding=embeddings
)

### **Realizando consultas no banco de dados vetorial**

In [ ]:
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

In [26]:
query = "What is Vector Store?"
vectorstore.similarity_search(query, k=1)

[Document(id='455ba684-4813-467e-9d39-8d00a4c9a12f', metadata={'source': 'data\\mediumblog1.txt'}, page_content='Title: Vector Database: What is it and Why You Should Know It?\n\nAuthor: Ejiro Onose\nDate: December 22, 2023\n\nIf 2021 was the year of graph databases, 2023 is the year of vector databases â€” Chip Huen.\n\nGenerative AI and Large Language Models (LLMs) have become popular, and a vector database is one of the best tools to handle LLM data. Vector databases provide the ideal infrastructure for managing the complex, high-dimensional data that LLMs produce and rely upon.\n\nIn this article, Iâ€™ll explain what vector databases are, how they work, and introduce some top vector database tools.\n\n What is a Vector?\nIn machine learning (ML), a vector is a collection of numerical values that represents the features of multi-dimensional objects, such as words or images. For example, a vector representing an image might contain values related to pixel intensities and color channe

### **CRIAR LLM**

In [27]:
llm = ChatGroq(
    model = "Gemma2-9b-It", # MODELO DE LLM UTILIZADO
    groq_api_key = GROQ_API_KEY, # CHAVE DE API DO GROQ
    temperature = 0.1, # TEMPERATURA DO LLM
)

#### **Carregar um prompt "padrão" via langchain hub** e criar o retriver

In [28]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain   = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

c:\Users\tavin\RAG_LLM_Introduction\.venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### **Criar a chain com base no prompt e retriver**

In [30]:
qa = create_retrieval_chain(
    vectorstore.as_retriever(),
    combine_docs_chain
)

In [31]:
res = qa.invoke({"input": "What is vector store in 3 setences?"})
print(res['answer']) 

A vector store is a specialized database designed to efficiently store and retrieve high-dimensional vectors. 

These vectors represent data points like words, images, or audio, capturing their semantic meaning or characteristics. 

Vector stores excel at similarity searches, allowing you to find data points that are closest to a given query vector. 



